In [29]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()

True

In [30]:
# 获取环境变量
deepseek_base_url = os.getenv("DEEPSEEK_BASE_URL")
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")

client = OpenAI(api_key=deepseek_api_key, base_url=deepseek_base_url)

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
    ],
    stream=False
)

print(response.choices[0].message.content)

Hello! How can I assist you today? 😊


In [40]:
# 获取环境变量
open_router_base_url = os.getenv("OPENROUTER_BASE_URL")
open_router_api_key = os.getenv("OPENROUTER_API_KEY")

client = OpenAI(base_url=open_router_base_url,api_key=open_router_api_key)

response = client.chat.completions.create(
    model="deepseek/deepseek-chat-v3-0324:free",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
    ],
    stream=False
)

print(response.choices[0].message.content)

Hello! How can I assist you today? 


In [72]:
system_prompt = '''请根据以下信息判断用户输入的数据是否符合电缆型号的标准，评判标准在<judgement></judgement>标签中。
<judgement>
电缆型号通常由以下部分组成：

用途代码：如Y（电力电缆）、K（控制电缆）、P（信号电缆）等。
绝缘材料代码：如Y（交联聚乙烯绝缘）、V（聚氯乙烯绝缘）。
护套材料代码：如P（聚氯乙烯护套）、V（聚乙烯护套）。
特殊性能代码：如Z（阻燃）、NH（耐火）。
导体材料代码：如T（铜芯）、L（铝芯）。
额定电压：如0.6/1kV、1/3kV等。
芯数及截面：如3*15+1x35平方毫米，表示3根15平方毫米导体芯线和1根35平方毫米导体芯线。

常见电缆型号示例:

YJV：交联聚乙烯绝缘聚氯乙烯护套电力电缆，适用于室内、隧道及管道中。
VV：聚氯乙烯绝缘和护套电力电缆，适用于室内、矿井中。
KVVP：控制电缆，适用于电器、仪表、配电装置的信号传输。
KVV：控制电缆，适用于室内、矿井中。
耐火电缆：如NH-YJV，具有耐火特性，适用于火灾环境。
阻燃电缆：如ZRA-YJV，具有阻燃特性，适用于普通环境。
截面与规格提取方法

根据型号中的数字部分，提取芯数和截面。例如，“495+150”表示4根95平方毫米导体芯线和1根50平方毫米导体芯线。
通过型号中的电压等级确定电缆的额定电压，如“0.6/1kV”表示额定电压为0.6kV，耐受电压为1kV。

训练数据示例

输入：WDZAYJV-0.6/1.0-4×95+1×50
输出：
型号：WDZAYJV
绝缘材料：交联聚乙烯绝缘
护套材料：聚氯乙烯护套
阻燃等级：A级阻燃
导体材料：铜芯
额定电压：0.6/1.0kV
芯数及截面：4根95平方毫米导体芯线和1根50平方毫米导体芯线。

注意事项
不同型号的电缆可能具有相同的用途代码或绝缘材料代码，需结合其他信息进行区分。

特殊性能代码（如耐火、阻燃）需优先识别，并根据具体需求标注。
</judgement>

根据以上评判标准，判断用户提供的数据是否符合电缆型号的标准。
用户提供的数据在<data></data>标签内。
请先step by step的进行分析，将分析结果写在<think></think>标签内，并给出最终结果放在<result></result>标签内。
注意结果标签只允许为以下两种格式，请选择其中一种：
<result>true</result>
或者
<result>false</result>
如果判断出属于符合电缆型号的标准后，在用户提供的<data-table></data-table>数据标准表中查找对应的设备编号。请step by step的进行分析，将分析结果写在<think></think>标签内，并给出最终对应设备编号结果放在<code></code>标签内。
如果没有在<data-table></data-table>中找到对应的设备编号，直接返回<code></code>空标签。
'''

prompt = '''124	配料室	屏蔽控制电缆	KVV-500V	KVVP-500  2*1.5	m	5600	0	0				
'''
prompt = '''113	配料室	焊接钢管DN15			m	30	0	0				
'''
prompt = '''126	配料室	屏蔽控制电缆	KVV-500V	KVV-500  5*1.0	m	10	0	0				
'''

In [ ]:
prompt='''
<data>129	配料室	屏蔽控制电缆	KVVP-500V	KVVP-500  10*1.5	m	60	0	0</data>
<data-table>
</data-table>
'''

In [74]:
def call_deepseek(base_url:str, api_key:str, prompt:str, system_prompt:str, model_name="deepseek-chat",temperature=0):
    try:
        client = OpenAI(api_key=api_key, base_url=base_url)

        response = client.chat.completions.create(
            model=model_name,
            temperature=temperature,       
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt},
            ],
            stream=False
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f'请求deepseek失败，原因：{e}')
        return None


In [88]:

result = call_deepseek(base_url=open_router_base_url,api_key=open_router_api_key,system_prompt=system_prompt,prompt=prompt,model_name='deepseek/deepseek-chat-v3-0324:free')

print(f'llm result:\n {result}')

llm result:
 <think>
1. 分析用户提供的电缆型号数据：
   - 型号：KVVP-500V
   - 规格：KVVP-500 10*1.5
   - 用途：屏蔽控制电缆
   - 电压等级：500V
   - 芯数及截面：10芯，1.5平方毫米

2. 检查是否符合电缆型号标准：
   - K：控制电缆
   - V：聚氯乙烯绝缘
   - V：聚氯乙烯护套
   - P：屏蔽
   - 500V：电压等级
   - 10*1.5：10芯1.5平方毫米
   符合电缆型号标准。

3. 在数据表中查找匹配项：
   - 需要查找控制电缆KVVP，10芯1.5平方毫米
   - 数据表中只有ZR-KVVP（阻燃）和通用控制电缆
   - 没有找到完全匹配的KVVP-500V 10*1.5
</think>

<result>true</result>
<code></code>
